Deltalake:
* Delta Lake is the optimized storage layer that provides the foundation for table

* Supported features

  - Schema enforcement and evolution
  - **Time travel (Data versoning)**
  - Data compaction (Optimize)
  - Unified Batch and Streaming Workloads
  - **Efficient Upserts and Deletes (MERGE operation)**
  - Scalability and Performance
  - Data Reliability and Checkpoints 
  - Compliance and Auditing 



Demo table:
- `customers` - Customer data with CDC history
- `products` - Product catalog
- `orders` - Order records
- `order_items` - Order line items
- `cdc_events` - Raw CDC events (audit log)

---
## 1. Setup and Configuration

In [47]:
# Configuration
DELTA_LAKE_PATH = "../deltalake"  # Delta Lake tables at project root (up one level from notebooks/)

# Available tables
TABLES = ["customers", "products", "orders", "order_items", "cdc_events"]

In [48]:
# Import libraries
import os

import pandas as pd
from deltalake import DeltaTable

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', None)

print("Libraries imported successfully!")
print(f"Delta Lake path: {os.path.abspath(DELTA_LAKE_PATH)}")

Libraries imported successfully!
Delta Lake path: /Users/anh.nguyen/Documents/poc/deltalake_poc/deltalake


In [49]:
# Helper functions
def get_table_path(table_name: str) -> str:
    """Get full path to Delta table."""
    return os.path.join(DELTA_LAKE_PATH, table_name)

def table_exists(table_name: str) -> bool:
    """Check if Delta table exists."""
    path = get_table_path(table_name)
    return os.path.exists(path) and os.path.exists(os.path.join(path, "_delta_log"))

def load_table(table_name: str, version: int = None) -> pd.DataFrame:
    """Load Delta table as pandas DataFrame."""
    path = get_table_path(table_name)
    if version is not None:
        dt = DeltaTable(path, version=version)
    else:
        dt = DeltaTable(path)
    return dt.to_pandas()

def get_history(table_name: str) -> list:
    """Get table history."""
    path = get_table_path(table_name)
    dt = DeltaTable(path)
    return dt.history()

def get_schema(table_name: str) -> dict:
    """Get table schema."""
    path = get_table_path(table_name)
    dt = DeltaTable(path)
    return dt.schema().to_pyarrow()

print("Helper functions defined!")

Helper functions defined!


---
## 2. Check Available Tables

In [50]:
# Check which tables exist
print("Available Delta Lake Tables:")
print("=" * 40)

for table in TABLES:
    exists = table_exists(table)
    status = "✅ Available" if exists else "❌ Not found"
    
    if exists:
        try:
            df = load_table(table)
            row_count = len(df)
            history = get_history(table)
            version_count = len(history)
            print(f"{table:15} {status} ({row_count} rows, {version_count} versions)")
        except Exception as e:
            print(f"{table:15} {status} (error: {e})")
    else:
        print(f"{table:15} {status}")

Available Delta Lake Tables:
customers       ✅ Available (error: Json error: whilst decoding field 'minValues': whilst decoding field 'created_at': failed to parse "+57947-06-07T01:15:05.000Z" as Timestamp(Microsecond, Some("UTC")): Parser error: Error parsing timestamp from '+57947-06-07T01:15:05.000Z': error parsing date)
products        ✅ Available (error: Json error: whilst decoding field 'minValues': whilst decoding field 'created_at': failed to parse "+57947-06-07T01:15:06.000Z" as Timestamp(Microsecond, Some("UTC")): Parser error: Error parsing timestamp from '+57947-06-07T01:15:06.000Z': error parsing date)
orders          ✅ Available (error: Json error: whilst decoding field 'minValues': whilst decoding field 'order_date': failed to parse "+57947-06-07T01:15:06.000Z" as Timestamp(Microsecond, Some("UTC")): Parser error: Error parsing timestamp from '+57947-06-07T01:15:06.000Z': error parsing date)
order_items     ✅ Available (error: Json error: whilst decoding field 'minValues

---
## 3. Query Current Data (delta-rs)

**Method:** Python-native `deltalake` library (delta-rs)  
**Pros:** Fast, lightweight, no JVM overhead  
**Cons:** Limited to basic operations (read, time travel, history)

In [51]:
# Query customers table
if table_exists("customers"):
    df_customers = load_table("customers")
    print(f"Customers Table ({len(df_customers)} rows)")
    print("=" * 60)
    display(df_customers)
else:
    print("Customers table not found. Run the CDC pipeline first.")

Exception: Json error: whilst decoding field 'minValues': whilst decoding field 'created_at': failed to parse "+57947-06-07T01:15:05.000Z" as Timestamp(Microsecond, Some("UTC")): Parser error: Error parsing timestamp from '+57947-06-07T01:15:05.000Z': error parsing date

In [52]:
# Query products table
if table_exists("products"):
    df_products = load_table("products")
    print(f"Products Table ({len(df_products)} rows)")
    print("=" * 60)
    display(df_products)
else:
    print("Products table not found. Run the CDC pipeline first.")

Exception: Json error: whilst decoding field 'minValues': whilst decoding field 'created_at': failed to parse "+57947-06-07T01:15:06.000Z" as Timestamp(Microsecond, Some("UTC")): Parser error: Error parsing timestamp from '+57947-06-07T01:15:06.000Z': error parsing date

In [13]:
# Query orders table
if table_exists("orders"):
    df_orders = load_table("orders")
    print(f"Orders Table ({len(df_orders)} rows)")
    print("=" * 60)
    display(df_orders)
else:
    print("Orders table not found. Run the CDC pipeline first.")

Orders Table (3 rows)


,id,customer_id,order_date,status,total_amount,shipping_address,created_at,updated_at,__cdc_operation,__cdc_timestamp,__processed_at
0,3,3,1970-01-21 10:41:11+00:00,pending,259.97,"789 Pine Rd, Chicago, IL 60601",1970-01-21 10:41:11+00:00,1970-01-21 10:41:11+00:00,r,2025-12-23 06:29:50+00:00,2025-12-23 06:43:20+00:00
1,2,2,1970-01-21 10:41:11+00:00,shipped,79.99,"456 Oak Ave, Los Angeles, CA 90001",1970-01-21 10:41:11+00:00,1970-01-21 10:41:11+00:00,r,2025-12-23 06:29:50+00:00,2025-12-23 06:43:19+00:00
2,1,1,1970-01-21 10:41:11+00:00,completed,1349.98,"123 Main St, New York, NY 10001",1970-01-21 10:41:11+00:00,1970-01-21 10:41:11+00:00,r,2025-12-23 06:29:50+00:00,2025-12-23 06:43:17+00:00


In [14]:
# Query CDC events (audit log)
if table_exists("cdc_events"):
    df_events = load_table("cdc_events")
    print(f"CDC Events ({len(df_events)} events)")
    print("=" * 60)
    display(df_events.head(20))
else:
    print("CDC events table not found. Run the CDC pipeline first.")

CDC Events (17 events)


,event_id,source_table,operation,record_id,before_data,after_data,kafka_topic,kafka_partition,kafka_offset,event_timestamp,processed_at
0,cdc.public.products-0-4,public.products,r,5,None,"{id: 5, name: Monitor Stand, description: Adju...",cdc.public.products,0,4,2025-12-23 06:29:50.604000+00:00,2025-12-23 06:43:26.687228+00:00
1,cdc.public.products-0-3,public.products,r,4,None,"{id: 4, name: Mechanical Keyboard, description...",cdc.public.products,0,3,2025-12-23 06:29:50.604000+00:00,2025-12-23 06:43:25.498653+00:00
2,cdc.public.products-0-2,public.products,r,3,None,"{id: 3, name: USB-C Hub, description: 7-in-1 U...",cdc.public.products,0,2,2025-12-23 06:29:50.603000+00:00,2025-12-23 06:43:24.396446+00:00
3,cdc.public.products-0-1,public.products,r,2,None,"{id: 2, name: Wireless Mouse, description: Erg...",cdc.public.products,0,1,2025-12-23 06:29:50.603000+00:00,2025-12-23 06:43:23.216876+00:00
4,cdc.public.products-0-0,public.products,r,1,None,"{id: 1, name: Laptop Pro, description: High-pe...",cdc.public.products,0,0,2025-12-23 06:29:50.603000+00:00,2025-12-23 06:43:21.824763+00:00
5,cdc.public.orders-0-2,public.orders,r,3,None,"{id: 3, customer_id: 3, order_date: 1766471296...",cdc.public.orders,0,2,2025-12-23 06:29:50.607000+00:00,2025-12-23 06:43:20.529542+00:00
6,cdc.public.orders-0-1,public.orders,r,2,None,"{id: 2, customer_id: 2, order_date: 1766471296...",cdc.public.orders,0,1,2025-12-23 06:29:50.607000+00:00,2025-12-23 06:43:18.864304+00:00
7,cdc.public.customers-0-4,public.customers,r,5,None,"{id: 5, first_name: Charlie, last_name: Brown,...",cdc.public.customers,0,4,2025-12-23 06:29:50.598000+00:00,2025-12-23 06:43:09.677926+00:00
8,cdc.public.customers-0-1,public.customers,r,2,None,"{id: 2, first_name: Jane, last_name: Smith, em...",cdc.public.customers,0,1,2025-12-23 06:29:50.597000+00:00,2025-12-23 06:43:04.313216+00:00
9,cdc.public.order_items-0-3,public.order_items,r,4,None,"{id: 4, order_id: 3, product_id: 4, quantity: ...",cdc.public.order_items,0,3,2025-12-23 06:29:50.610000+00:00,2025-12-23 06:43:15.237082+00:00


In [16]:
# Detailed version analysis - see what changed in each version
TABLE_TO_ANALYZE = "customers"

if table_exists(TABLE_TO_ANALYZE):
    print(f"=== Detailed Version Analysis: {TABLE_TO_ANALYZE} ===\n")
    
    history = get_history(TABLE_TO_ANALYZE)
    
    print(f"Total versions: {len(history)}\n")
    print("Version Details:")
    print("=" * 100)
    
    for entry in history[:15]:  # Show last 15 versions
        version = entry.get('version', 'N/A')
        timestamp = entry.get('timestamp', 'N/A')
        operation = entry.get('operation', 'N/A')
        
        # Get operation metrics if available
        metrics = entry.get('operationMetrics', {})
        num_output_rows = metrics.get('numOutputRows', 'N/A')
        num_updated_rows = metrics.get('numTargetRowsUpdated', 'N/A')
        num_inserted_rows = metrics.get('numTargetRowsInserted', 'N/A')
        
        print(f"v{version:2} | {timestamp} | {operation:15} | Rows: out={num_output_rows}, updated={num_updated_rows}, inserted={num_inserted_rows}")
    
    print("\n" + "=" * 100)
    print("\nKey Metrics:")
    print("- 'numOutputRows': Total rows after operation")
    print("- 'numTargetRowsUpdated': Rows updated by MERGE")
    print("- 'numTargetRowsInserted': Rows inserted by MERGE")
else:
    print(f"Table '{TABLE_TO_ANALYZE}' not found.")

=== Detailed Version Analysis: customers ===

Total versions: 6

Version Details:
v 5 | 1766472191050 | MERGE           | Rows: out=1, updated=0, inserted=1
v 4 | 1766472189552 | MERGE           | Rows: out=1, updated=0, inserted=1
v 3 | 1766472187860 | MERGE           | Rows: out=1, updated=0, inserted=1
v 2 | 1766472186208 | MERGE           | Rows: out=1, updated=0, inserted=1
v 1 | 1766472184122 | MERGE           | Rows: out=1, updated=0, inserted=1
v 0 | 1766472180826 | CREATE TABLE    | Rows: out=N/A, updated=N/A, inserted=N/A


Key Metrics:
- 'numOutputRows': Total rows after operation
- 'numTargetRowsUpdated': Rows updated by MERGE
- 'numTargetRowsInserted': Rows inserted by MERGE


In [17]:
# Compare row count across versions to see when records were added
TABLE_TO_ANALYZE = "customers"

if table_exists(TABLE_TO_ANALYZE):
    print(f"=== Row Count Evolution: {TABLE_TO_ANALYZE} ===\n")
    
    history = get_history(TABLE_TO_ANALYZE)
    
    for entry in history[:15]:
        version = entry.get('version', 'N/A')
        
        try:
            df = load_table(TABLE_TO_ANALYZE, version=version)
            row_count = len(df)
            
            # Get operation info
            operation = entry.get('operation', 'N/A')
            timestamp = entry.get('timestamp', 'N/A')
            
            print(f"Version {version:2} | {operation:15} | {row_count} rows | {timestamp}")
        except Exception as e:
            print(f"Version {version:2} | Error: {e}")
    
    print("\n💡 Observation:")
    print("   - If row count doesn't change between versions, that MERGE was an UPDATE")
    print("   - If row count increases, that MERGE added a new record")
else:
    print(f"Table '{TABLE_TO_ANALYZE}' not found.")

=== Row Count Evolution: customers ===

Version  5 | MERGE           | 5 rows | 1766472191050
Version  4 | MERGE           | 4 rows | 1766472189552
Version  3 | MERGE           | 3 rows | 1766472187860
Version  2 | MERGE           | 2 rows | 1766472186208
Version  1 | MERGE           | 1 rows | 1766472184122
Version  0 | CREATE TABLE    | 0 rows | 1766472180826

💡 Observation:
   - If row count doesn't change between versions, that MERGE was an UPDATE
   - If row count increases, that MERGE added a new record


In [18]:
# Time travel: Query specific version
TABLE_TO_EXPLORE = "customers"
VERSION = 0 

if table_exists(TABLE_TO_EXPLORE):
    try:
        df_historical = load_table(TABLE_TO_EXPLORE, version=VERSION)
        print(f"'{TABLE_TO_EXPLORE}' at Version {VERSION}")
        print("=" * 60)
        display(df_historical)
    except Exception as e:
        print(f"Error loading version {VERSION}: {e}")
else:
    print(f"Table '{TABLE_TO_EXPLORE}' not found.")

'customers' at Version 0


,id,first_name,last_name,email,phone,created_at,updated_at,__cdc_operation,__cdc_timestamp,__processed_at


In [19]:
# Compare two versions
TABLE_TO_COMPARE = "customers"
VERSION_OLD = 0
VERSION_NEW = 4  # None = latest

if table_exists(TABLE_TO_COMPARE):
    try:
        df_old = load_table(TABLE_TO_COMPARE, version=VERSION_OLD)
        df_new = load_table(TABLE_TO_COMPARE, version=VERSION_NEW)
        
        print(f"Version {VERSION_OLD}: {len(df_old)} rows")
        print(f"Latest version: {len(df_new)} rows")
        
        # Show side by side if small enough
        if len(df_old) <= 10 and len(df_new) <= 10:
            print(f"\n------------------------------- Version {VERSION_OLD} -------------------------------")
            display(df_old)
            print("\n------------------------------- Latest -------------------------------")
            display(df_new)
    except Exception as e:
        print(f"Error comparing versions: {e}")
else:
    print(f"Table '{TABLE_TO_COMPARE}' not found.")

Version 0: 0 rows
Latest version: 4 rows

------------------------------- Version 0 -------------------------------


,id,first_name,last_name,email,phone,created_at,updated_at,__cdc_operation,__cdc_timestamp,__processed_at



------------------------------- Latest -------------------------------


,id,first_name,last_name,email,phone,created_at,updated_at,__cdc_operation,__cdc_timestamp,__processed_at
0,4,Alice,Williams,alice.williams@example.com,+1-555-0104,1970-01-21 10:41:11+00:00,1970-01-21 10:41:11+00:00,r,2025-12-23 06:29:50+00:00,2025-12-23 06:43:08+00:00
1,3,Bob,Johnson,bob.johnson@example.com,+1-555-0103,1970-01-21 10:41:11+00:00,1970-01-21 10:41:11+00:00,r,2025-12-23 06:29:50+00:00,2025-12-23 06:43:06+00:00
2,2,Jane,Smith,jane.smith@example.com,+1-555-0102,1970-01-21 10:41:11+00:00,1970-01-21 10:41:11+00:00,r,2025-12-23 06:29:50+00:00,2025-12-23 06:43:04+00:00
3,1,John,Doe,john.doe@example.com,+1-555-0101,1970-01-21 10:41:11+00:00,1970-01-21 10:41:11+00:00,r,2025-12-23 06:29:50+00:00,2025-12-23 06:43:00+00:00


In [20]:
# Analyze CDC events by operation type
if table_exists("cdc_events"):
    df_events = load_table("cdc_events")
    
    print("CDC Events by Operation Type")
    print("=" * 40)
    
    operation_counts = df_events['operation'].value_counts()
    operation_map = {'c': 'CREATE', 'u': 'UPDATE', 'd': 'DELETE', 'r': 'READ (snapshot)'}
    
    for op, count in operation_counts.items():
        op_name = operation_map.get(op, op)
        print(f"{op_name:20} {count:5} events")
else:
    print("CDC events table not found.")

CDC Events by Operation Type
READ (snapshot)         17 events


In [21]:
# Analyze CDC events by source table
if table_exists("cdc_events"):
    df_events = load_table("cdc_events")
    
    print("CDC Events by Source Table")
    print("=" * 40)
    
    table_counts = df_events['source_table'].value_counts()
    for table, count in table_counts.items():
        print(f"{table:30} {count:5} events")
else:
    print("CDC events table not found.")

CDC Events by Source Table
public.products                    5 events
public.customers                   5 events
public.order_items                 4 events
public.orders                      3 events


In [22]:
# View recent CDC events
if table_exists("cdc_events"):
    df_events = load_table("cdc_events")
    
    print("Recent CDC Events (last 10)")
    print("=" * 60)
    
    # Sort by processed_at if available, otherwise by event_id
    if 'processed_at' in df_events.columns:
        df_recent = df_events.sort_values('processed_at', ascending=False).head(10)
    else:
        df_recent = df_events.tail(10)
    
    display(df_recent[['event_id', 'source_table', 'operation', 'record_id', 'event_timestamp']])
else:
    print("CDC events table not found.")

Recent CDC Events (last 10)


,event_id,source_table,operation,record_id,event_timestamp
0,cdc.public.products-0-4,public.products,r,5,2025-12-23 06:29:50.604000+00:00
1,cdc.public.products-0-3,public.products,r,4,2025-12-23 06:29:50.604000+00:00
2,cdc.public.products-0-2,public.products,r,3,2025-12-23 06:29:50.603000+00:00
3,cdc.public.products-0-1,public.products,r,2,2025-12-23 06:29:50.603000+00:00
4,cdc.public.products-0-0,public.products,r,1,2025-12-23 06:29:50.603000+00:00
5,cdc.public.orders-0-2,public.orders,r,3,2025-12-23 06:29:50.607000+00:00
6,cdc.public.orders-0-1,public.orders,r,2,2025-12-23 06:29:50.607000+00:00
11,cdc.public.orders-0-0,public.orders,r,1,2025-12-23 06:29:50.607000+00:00
9,cdc.public.order_items-0-3,public.order_items,r,4,2025-12-23 06:29:50.610000+00:00
16,cdc.public.order_items-0-2,public.order_items,r,3,2025-12-23 06:29:50.609000+00:00


---
## 4. PySpark SQL Queries

**Method:** PySpark with Delta Lake SQL  
**Pros:** Full SQL support, complex transformations, aggregations  
**Cons:** Requires JVM, heavier resource usage  
**Python 3.14 Note:** Use SQL-based operations, avoid `createDataFrame()` due to serialization issues

### Read

In [53]:
# Initialize PySpark with Delta Lake
USE_SPARK = True  # Set to True to enable PySpark

if USE_SPARK:
    from pyspark.sql import SparkSession
    from delta import configure_spark_with_delta_pip

    builder = (
        SparkSession.builder
        .appName("DeltaLakeNotebook")
        .master("local[*]")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    )
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
    print("SparkSession created!")
    print(f"Spark version: {spark.version}")
else:
    print("PySpark disabled. Set USE_SPARK = True to enable.")

SparkSession created!
Spark version: 3.5.0


In [54]:
# PySpark SQL: Query with SQL
if USE_SPARK and 'spark' in dir():
    # Load Delta tables
    customers_path = get_table_path("customers")
    
    if table_exists("customers"):
        # Method 1: Read as DataFrame
        df_spark = spark.read.format("delta").load(customers_path)
        df_spark.createOrReplaceTempView("customers")
        
        # Method 2: Run SQL query
        print("------------------------ PySpark SQL Query ------------------------ ")
        result = spark.sql("""
            SELECT *
            FROM customers
            ORDER BY id
        """)
        result.show()
        
        print(f"\nTotal customers: {result.count()}")
    else:
        print("Customers table not found.")
else:
    print("Spark not enabled or customers table not found.")

------------------------ PySpark SQL Query ------------------------ 
+---+----------+---------+--------------------+-----------+--------------------+--------------------+---------------+--------------------+--------------------+
| id|first_name|last_name|               email|      phone|          created_at|          updated_at|__cdc_operation|     __cdc_timestamp|      __processed_at|
+---+----------+---------+--------------------+-----------+--------------------+--------------------+---------------+--------------------+--------------------+
|  1|      John|      Doe|john.doe@example.com|+1-555-0101|+57947-06-07 08:1...|+57947-06-07 08:1...|              r|2025-12-23 15:05:...|2025-12-23 15:06:...|
|  2|      Jane|    Smith|jane.smith@exampl...|+1-555-0102|+57947-06-07 08:1...|+57947-06-07 08:1...|              r|2025-12-23 15:05:...|2025-12-23 15:06:...|
|  3|       Bob|  Johnson|bob.johnson@examp...|+1-555-0103|+57947-06-07 08:1...|+57947-06-07 08:1...|              r|2025-12-23 15:

In [59]:
# PySpark SQL: Time travel query
if USE_SPARK and 'spark' in dir():
    customers_path = get_table_path("customers")
    
    if table_exists("customers"):
        print("------------------------  PySpark Time Travel ------------------------\n")
        
        # Query version 0 (first snapshot)
        print("Version 0 (Initial Snapshot):")
        df_v0 = spark.read.format("delta").option("versionAsOf", 0).load(customers_path)
        df_v0.show()
        
        # Query latest version
        print("\nLatest Version:")
        df_latest = spark.read.format("delta").option("versionAsOf", 1).load(customers_path)
        df_latest.show()
        
        print(f"\nVersion 0 rows: {df_v0.count()}")
        print(f"Latest rows: {df_latest.count()}")
    else:
        print("Customers table not found.")
else:
    print("Spark not enabled or customers table not found.")

------------------------  PySpark Time Travel ------------------------

Version 0 (Initial Snapshot):
+---+----------+---------+--------------------+-----------+--------------------+--------------------+---------------+--------------------+--------------------+
| id|first_name|last_name|               email|      phone|          created_at|          updated_at|__cdc_operation|     __cdc_timestamp|      __processed_at|
+---+----------+---------+--------------------+-----------+--------------------+--------------------+---------------+--------------------+--------------------+
|  1|      John|      Doe|john.doe@example.com|+1-555-0101|+57947-06-07 08:1...|+57947-06-07 08:1...|              r|2025-12-23 15:05:...|2025-12-23 15:06:...|
|  2|      Jane|    Smith|jane.smith@exampl...|+1-555-0102|+57947-06-07 08:1...|+57947-06-07 08:1...|              r|2025-12-23 15:05:...|2025-12-23 15:06:...|
|  3|       Bob|  Johnson|bob.johnson@examp...|+1-555-0103|+57947-06-07 08:1...|+57947-06-07 08:1.

In [60]:
# PySpark SQL: Advanced aggregations and filtering
if USE_SPARK and 'spark' in dir() and table_exists("customers"):
    customers_path = get_table_path("customers")
    df = spark.read.format("delta").load(customers_path)
    df.createOrReplaceTempView("customers")
    
    print("------------------------------- Advanced SQL Queries -------------------------------\n")
    
    print("1. Group by CDC operation:")
    spark.sql("""
        SELECT __cdc_operation, COUNT(*) as event_count
        FROM customers
        GROUP BY __cdc_operation
        ORDER BY event_count DESC
    """).show()
    
    print("\n2. Filter and sort customers:")
    spark.sql("""
        SELECT first_name, last_name, email, phone
        FROM customers
        ORDER BY first_name, last_name
        LIMIT 5
    """).show()
    
    print("\n3. Recent changes (by CDC timestamp):")
    spark.sql("""
        SELECT first_name, last_name, __cdc_operation, __cdc_timestamp
        FROM customers
        ORDER BY __cdc_timestamp DESC
        LIMIT 5
    """).show()
else:
    print("Spark not enabled or customers table not found.")

------------------------------- Advanced SQL Queries -------------------------------

1. Group by CDC operation:
+---------------+-----------+
|__cdc_operation|event_count|
+---------------+-----------+
|              r|          5|
|              c|          1|
|              u|          1|
+---------------+-----------+


2. Filter and sort customers:
+----------+---------+--------------------+-----------+
|first_name|last_name|               email|      phone|
+----------+---------+--------------------+-----------+
|     Alice| Williams|alice.williams@ex...|+1-555-0104|
|       Bob|  Johnson|bob.johnson@examp...|+1-555-0103|
|   Charlie|    Brown|charlie.brown@exa...|+1-555-0105|
|      Jane|    Smith|jane.smith@exampl...|+1-555-0102|
|      John|      Doe|john.doe@example.com|+1-555-0101|
+----------+---------+--------------------+-----------+


3. Recent changes (by CDC timestamp):
+----------+---------+---------------+--------------------+
|first_name|last_name|__cdc_operation|   

In [61]:
# PySpark SQL: View Delta table history (on actual CDC tables)
if USE_SPARK and 'spark' in dir() and table_exists("customers"):
    from delta import DeltaTable
    
    customers_path = get_table_path("customers")
    
    print("------------------------------- Delta Table History (Real CDC Data) -------------------------------\n")
    print("Showing version history for 'customers' table:\n")
    
    dt = DeltaTable.forPath(spark, customers_path)
    dt.history().select("version", "timestamp", "operation", "operationMetrics").show(truncate=False)
else:
    print("Spark not enabled or customers table not found.")

------------------------------- Delta Table History (Real CDC Data) -------------------------------

Showing version history for 'customers' table:

+-------+-----------------------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|timestamp              |operation|operationMetrics                                                                                                                       

In [62]:
# PySpark: CDC Events Analysis
if USE_SPARK and 'spark' in dir() and table_exists("cdc_events"):
    events_path = get_table_path("cdc_events")
    
    df_events = spark.read.format("delta").load(events_path)
    df_events.createOrReplaceTempView("cdc_events")
    
    print("=== CDC Events Timeline Analysis ===\n")
    
    print("1. Events by operation type:")
    spark.sql("""
        SELECT 
            operation,
            COUNT(*) as event_count,
            MIN(event_timestamp) as first_event,
            MAX(event_timestamp) as last_event
        FROM cdc_events
        GROUP BY operation
        ORDER BY event_count DESC
    """).show()
    
    print("\n2. Events by source table:")
    spark.sql("""
        SELECT 
            source_table,
            operation,
            COUNT(*) as count
        FROM cdc_events
        GROUP BY source_table, operation
        ORDER BY source_table, count DESC
    """).show()
else:
    print("Spark not enabled or cdc_events table not found.")

=== CDC Events Timeline Analysis ===

1. Events by operation type:
+---------+-----------+--------------------+--------------------+
|operation|event_count|         first_event|          last_event|
+---------+-----------+--------------------+--------------------+
|        r|         17|2025-12-23 15:05:...|2025-12-23 15:05:...|
|        c|          3|2025-12-23 15:11:...|2025-12-23 15:11:...|
|        u|          2|2025-12-23 15:11:...|2025-12-23 15:11:...|
+---------+-----------+--------------------+--------------------+


2. Events by source table:
+------------------+---------+-----+
|      source_table|operation|count|
+------------------+---------+-----+
|  public.customers|        r|    5|
|  public.customers|        u|    1|
|  public.customers|        c|    1|
|public.order_items|        r|    4|
|     public.orders|        r|    3|
|     public.orders|        c|    1|
|   public.products|        r|    5|
|   public.products|        c|    1|
|   public.products|        u|    1

In [63]:
# PySpark: Compare versions of customers table (Time Travel)
if USE_SPARK and 'spark' in dir() and table_exists("customers"):
    customers_path = get_table_path("customers")
    
    print("=== Time Travel: Compare Customer Versions ===\n")
    
    try:
        # Get version count
        from delta import DeltaTable
        dt = DeltaTable.forPath(spark, customers_path)
        history = dt.history().select("version").collect()
        max_version = max([row.version for row in history])
        
        print(f"Available versions: 0 to {max_version}\n")
        
        # Compare version 0 vs latest
        print("----------------------------- Version 0 (Initial) -----------------------------")
        df_v0 = spark.read.format("delta").option("versionAsOf", 0).load(customers_path)
        print(f"Row count: {df_v0.count()}")
        df_v0.show(5)
        
        print(f"\n----------------------------- Latest Version (v{max_version}) -----------------------------")
        df_v6 = spark.read.format("delta").option("versionAsOf", 6).load(customers_path)
        print(f"Row count: {df_v0.count()}")
        df_v6.show(5)
        
    except Exception as e:
        print(f"Error: {e}")
else:
    print("Spark not enabled or customers table not found.")

=== Time Travel: Compare Customer Versions ===

Available versions: 0 to 1

----------------------------- Version 0 (Initial) -----------------------------
Row count: 5
+---+----------+---------+--------------------+-----------+--------------------+--------------------+---------------+--------------------+--------------------+
| id|first_name|last_name|               email|      phone|          created_at|          updated_at|__cdc_operation|     __cdc_timestamp|      __processed_at|
+---+----------+---------+--------------------+-----------+--------------------+--------------------+---------------+--------------------+--------------------+
|  1|      John|      Doe|john.doe@example.com|+1-555-0101|+57947-06-07 08:1...|+57947-06-07 08:1...|              r|2025-12-23 15:05:...|2025-12-23 15:06:...|
|  2|      Jane|    Smith|jane.smith@exampl...|+1-555-0102|+57947-06-07 08:1...|+57947-06-07 08:1...|              r|2025-12-23 15:05:...|2025-12-23 15:06:...|
|  3|       Bob|  Johnson|bob.j

In [65]:
from delta import DeltaTable

customers_path = get_table_path("customers")

print("=== Time Travel by Timestamp: Version 5 vs Version 6 ===\n")

# Load Delta table
dt = DeltaTable.forPath(spark, customers_path)

# Get history with timestamps
history_df = (
    dt.history(10)
      .select("version", "timestamp", "operation")
      .orderBy("version")
)

# history_df.show(truncate=False)

# Extract timestamps for v5 and v6
history = history_df.collect()

ts_v5 = next(row.timestamp for row in history if row.version == 0)
ts_v6 = next(row.timestamp for row in history if row.version == 1)

print(f"Version 0 timestamp: {ts_v5}")
print(f"Version 1 timestamp: {ts_v6}\n")

# Read data AS OF version 0 (by timestamp)
df_v5 = (
    spark.read.format("delta")
    .option("timestampAsOf", ts_v5)
    .load(customers_path)
)

# Read data AS OF version 6 (by timestamp)
df_v6 = (
    spark.read.format("delta")
    .option("timestampAsOf", ts_v6)
    .load(customers_path)
)

# Compare
print("----------------------------- Version 0 -----------------------------")
print(f"Row count: {df_v5.count()}")
df_v5.show(5, truncate=False)

print("----------------------------- Version 1 -----------------------------")
print(f"Row count: {df_v6.count()}")
df_v6.show(5, truncate=False)


=== Time Travel by Timestamp: Version 5 vs Version 6 ===

Version 0 timestamp: 2025-12-23 15:06:29.640000
Version 1 timestamp: 2025-12-23 15:11:58.029000

----------------------------- Version 0 -----------------------------
Row count: 5
+---+----------+---------+--------------------------+-----------+---------------------+---------------------+---------------+-----------------------+-----------------------+
|id |first_name|last_name|email                     |phone      |created_at           |updated_at           |__cdc_operation|__cdc_timestamp        |__processed_at         |
+---+----------+---------+--------------------------+-----------+---------------------+---------------------+---------------+-----------------------+-----------------------+
|1  |John      |Doe      |john.doe@example.com      |+1-555-0101|+57947-06-07 08:15:05|+57947-06-07 08:15:05|r              |2025-12-23 15:05:29.156|2025-12-23 15:06:21.757|
|2  |Jane      |Smith    |jane.smith@example.com    |+1-555-0102|+

In [66]:
# Track all changes on a specific customer record using Change Data Feed
if USE_SPARK and 'spark' in dir() and table_exists("customers"):
    from delta import DeltaTable
    
    customers_path = get_table_path("customers")
    
    try:
        # Enable Change Data Feed on the table (if not already enabled)
        dt = DeltaTable.forPath(spark, customers_path)
        
        # all changes (insert, update, delete) for each record
        changes_df = (
            spark.read.format("delta")
            .option("readChangeFeed", "true")
            .option("startingVersion", 0)
            .load(customers_path)
        )
        
        # changes on a specific customer (ID = 1)
        customer_id = 1
        customer_changes = changes_df.filter(f"id = {customer_id}").orderBy("_commit_version")
        
        # Show key columns including change metadata
        customer_changes.select(
            "id", 
            "first_name", 
            "last_name", 
            "email",
            "_change_type",      # insert, update_preimage, update_postimage, delete
            "_commit_version",   # Delta version number
            "_commit_timestamp"  # When change occurred
        ).show(truncate=False)
        
        print(f"\nTotal changes tracked: {customer_changes.count()}")
        
    except Exception as e:
        print(f"Error: {e}")
else:
    print("Spark not enabled or customers table not found.")

+---+----------+---------+--------------------+------------+---------------+----------------------+
|id |first_name|last_name|email               |_change_type|_commit_version|_commit_timestamp     |
+---+----------+---------+--------------------+------------+---------------+----------------------+
|1  |John      |Doe      |john.doe@example.com|insert      |0              |2025-12-23 15:06:29.64|
+---+----------+---------+--------------------+------------+---------------+----------------------+


Total changes tracked: 1


In [67]:
if USE_SPARK and 'spark' in dir() and table_exists("customers"):
    
    customers_path = get_table_path("customers")
    
    try:
        changes_df = (
            spark.read.format("delta")
            .option("readChangeFeed", "true")
            .option("startingVersion", 0)
            .load(customers_path)
        )
        
        customer_id = 1
        
        final_states = changes_df.filter(
            f"id = {customer_id} AND _change_type IN ('insert', 'update_postimage', 'delete')"
        ).orderBy("_commit_version")
        
        print(f"Customer ID {customer_id} - Final States Only (cleaner view):")
        print("=" * 100)
        
        final_states.select(
            "_commit_version",
            "_change_type",
            "id", 
            "first_name", 
            "last_name", 
            "email",
            "_commit_timestamp"
        ).show(truncate=False)
        
        print(f"\n✓ This shows only the 'after' state of each change")
        print(f"  Total state changes: {final_states.count()}")
        
    except Exception as e:
        print(f"Error: {e}")
else:
    print("Spark not enabled or customers table not found.")

Customer ID 1 - Final States Only (cleaner view):
+---------------+------------+---+----------+---------+--------------------+----------------------+
|_commit_version|_change_type|id |first_name|last_name|email               |_commit_timestamp     |
+---------------+------------+---+----------+---------+--------------------+----------------------+
|0              |insert      |1  |John      |Doe      |john.doe@example.com|2025-12-23 15:06:29.64|
+---------------+------------+---+----------+---------+--------------------+----------------------+


✓ This shows only the 'after' state of each change
  Total state changes: 1


In [68]:
from delta import DeltaTable

dt = DeltaTable.forPath(spark, customers_path)
history_df = dt.history()

history_df.show(truncate=False)


+-------+-----------------------+------+--------+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp   

In [69]:
# PySpark: Product analysis with order items
if USE_SPARK and 'spark' in dir() and table_exists("products") and table_exists("order_items"):
    products_path = get_table_path("products")
    items_path = get_table_path("order_items")
    
    df_products = spark.read.format("delta").load(products_path)
    df_items = spark.read.format("delta").load(items_path)
    
    df_products.createOrReplaceTempView("products")
    df_items.createOrReplaceTempView("order_items")
    
    print("----------------------------- Product Popularity: Most Ordered Items -----------------------------\n")
    
    spark.sql("""
        SELECT 
            p.name as product_name,
            p.category,
            COUNT(oi.id) as times_ordered,
            SUM(oi.quantity) as total_quantity_sold
        FROM products p
        LEFT JOIN order_items oi ON p.id = oi.product_id
        GROUP BY p.id, p.name, p.category
        ORDER BY total_quantity_sold DESC
        LIMIT 10
    """).show()
else:
    print("Spark not enabled or tables not found.")

----------------------------- Product Popularity: Most Ordered Items -----------------------------

+-------------------+-----------+-------------+-------------------+
|       product_name|   category|times_ordered|total_quantity_sold|
+-------------------+-----------+-------------+-------------------+
|Mechanical Keyboard|Electronics|            1|                  2|
|          USB-C Hub|Electronics|            1|                  1|
|         Laptop Pro|Electronics|            1|                  1|
|     Wireless Mouse|Electronics|            1|                  1|
|      Monitor Stand|     Office|            0|               NULL|
|       Test Product|       Test|            0|               NULL|
+-------------------+-----------+-------------+-------------------+



In [39]:
# PySpark: Analyze order details
if USE_SPARK and 'spark' in dir() and table_exists("orders") and table_exists("order_items"):
    orders_path = get_table_path("orders")
    items_path = get_table_path("order_items")
    
    df_orders = spark.read.format("delta").load(orders_path)
    df_items = spark.read.format("delta").load(items_path)
    
    df_orders.createOrReplaceTempView("orders")
    df_items.createOrReplaceTempView("order_items")
    
    print("----------------------------- Order Analysis: Items per Order -----------------------------\n")
    
    spark.sql("""
        SELECT 
            o.id as order_id,
            o.customer_id,
            o.order_date,
            COUNT(oi.id) as item_count,
            SUM(oi.quantity) as total_quantity
        FROM orders o
        LEFT JOIN order_items oi ON o.id = oi.order_id
        GROUP BY o.id, o.customer_id, o.order_date
        ORDER BY total_quantity DESC
        LIMIT 10
    """).show()
else:
    print("Spark not enabled or tables not found.")

----------------------------- Order Analysis: Items per Order -----------------------------

+--------+-----------+-------------------+----------+--------------+
|order_id|customer_id|         order_date|item_count|total_quantity|
+--------+-----------+-------------------+----------+--------------+
|       1|          1|1970-01-21 18:41:11|         2|             2|
|       3|          3|1970-01-21 18:41:11|         1|             2|
|       2|          2|1970-01-21 18:41:11|         1|             1|
+--------+-----------+-------------------+----------+--------------+



In [40]:
# PySpark: Join customers with orders
if USE_SPARK and 'spark' in dir() and table_exists("customers") and table_exists("orders"):
    # Load both tables
    customers_path = get_table_path("customers")
    orders_path = get_table_path("orders")
    
    df_customers = spark.read.format("delta").load(customers_path)
    df_orders = spark.read.format("delta").load(orders_path)
    
    df_customers.createOrReplaceTempView("customers")
    df_orders.createOrReplaceTempView("orders")
    
    print("=== JOIN: Customers with Their Orders ===\n")
    
    spark.sql("""
        SELECT 
            c.id as customer_id,
            c.first_name,
            c.last_name,
            c.email,
            COUNT(o.id) as total_orders
        FROM customers c
        LEFT JOIN orders o ON c.id = o.customer_id
        GROUP BY c.id, c.first_name, c.last_name, c.email
        ORDER BY total_orders DESC
    """).show()
else:
    print("Spark not enabled or tables not found.")

=== JOIN: Customers with Their Orders ===

+-----------+----------+---------+--------------------+------------+
|customer_id|first_name|last_name|               email|total_orders|
+-----------+----------+---------+--------------------+------------+
|          3|       Bob|  Johnson|bob.johnson@examp...|           1|
|          2|      Jane|    Smith|jane.smith@exampl...|           1|
|          1|      John|      Doe|john.doe@example.com|           1|
|          4|     Alice| Williams|alice.williams@ex...|           0|
|          5|   Charlie|    Brown|charlie.brown@exa...|           0|
+-----------+----------+---------+--------------------+------------+



### Write

In [139]:
# Demo: Create a new Delta table

demo_table_path = "/Users/anh.nguyen/Documents/poc/deltalake_poc/deltalake/demo_employees"
demo_table_name = "demo_employees"

if USE_SPARK and 'spark' in dir():
    
    print("=== Demo: Create Table and Insert Records ===\n")
    
    try:
        # Drop table if exist
        spark.sql(f"""
                DROP TABLE IF EXISTS {demo_table_name}
                  """)    

        # 1: Create new Delta table
        print("Step 1: Creating new Delta table...")
        spark.sql(f"""
            CREATE TABLE IF NOT EXISTS {demo_table_name} (
                id BIGINT NOT NULL,
                name STRING NOT NULL,
                department STRING,
                salary DECIMAL(10, 2),
                hire_date DATE,
                is_active BOOLEAN
            ) USING DELTA
            LOCATION '{demo_table_path}'
        """)
        print("Table created!\n")
        
    except Exception as e:
        print(f"Error: {e}")
else:
    print("Spark not enabled. Set USE_SPARK = True in Section 6.")

=== Demo: Create Table and Insert Records ===

Step 1: Creating new Delta table...
Table created!



In [140]:
# Demo: Insert records using PySpark SQL
if USE_SPARK and 'spark' in dir():
        
    try:        
        # 2: Insert initial records
        print("2: Inserting initial records...")
        spark.sql(f"""
            INSERT INTO {demo_table_name} VALUES
            (1, 'Alice Johnson', 'Engineering', 95000.00, DATE '2020-01-15', true),
            (2, 'Bob Smith', 'Marketing', 75000.00, DATE '2021-03-20', true),
            (3, 'Charlie Brown', 'Engineering', 88000.00, DATE '2019-07-10', true)
        """)
        print("3 records inserted!\n")
        
    except Exception as e:
        print(f"Error: {e}")
else:
    print("Spark not enabled. Set USE_SPARK = True in Section 6.")

2: Inserting initial records...
3 records inserted!



In [141]:
if USE_SPARK and 'spark' in dir():

    try:                
        # 3: View the data
        print("Step 3: Viewing inserted data:")
        spark.sql(f"SELECT * FROM {demo_table_name}").show()
        
    except Exception as e:
        print(f"Error: {e}")
else:
    print("Spark not enabled. Set USE_SPARK = True in Section 6.")

Step 3: Viewing inserted data:
+---+-------------+------------+---------+----------+---------+
| id|         name|  department|   salary| hire_date|is_active|
+---+-------------+------------+---------+----------+---------+
|  3|Charlie Brown| Engineering| 88000.00|2019-07-10|     true|
|  1|Alice Johnson| Engineering|105000.00|2020-01-15|     true|
|  1|Alice Johnson| Engineering| 95000.00|2020-01-15|     true|
|  5| Eve Anderson| Engineering| 92000.00|2021-11-08|     true|
|  3|Charlie Brown|Data Science| 98000.00|2019-07-10|     true|
|  6| Frank Miller|       Sales| 78000.00|2023-01-15|     true|
|  2|    Bob Smith|   Marketing| 75000.00|2021-03-20|     true|
|  2|    Bob Smith|   Marketing| 75000.00|2021-03-20|    false|
|  4| Diana Prince|       Sales| 82000.00|2022-05-12|     true|
+---+-------------+------------+---------+----------+---------+



In [103]:
if USE_SPARK and 'spark' in dir():
    
    try:        
        # 4: Insert more records
        print("\n4: Inserting additional records...")
        spark.sql(f"""
            INSERT INTO {demo_table_name} VALUES
            (4, 'Diana Prince', 'Sales', 82000.00, DATE '2022-05-12', true),
            (5, 'Eve Anderson', 'Engineering', 92000.00, DATE '2021-11-08', true)
        """)
        print("2 more records inserted!\n")
        
    except Exception as e:
        print(f"Error: {e}")
else:
    print("Spark not enabled. Set USE_SPARK = True in Section 6.")


4: Inserting additional records...
2 more records inserted!



In [104]:
if USE_SPARK and 'spark' in dir():
    
    try:        
        # 5: View updated data
        print("Step 5: Viewing all records:")
        result = spark.sql(f"SELECT * FROM {demo_table_name} ORDER BY id")
        print(f"Total records: {result.count()}")
        result.show()

    except Exception as e:
        print(f"Error: {e}")
else:
    print("Spark not enabled. Set USE_SPARK = True in Section 6.")

Step 5: Viewing all records:
Total records: 5
+---+-------------+-----------+--------+----------+---------+
| id|         name| department|  salary| hire_date|is_active|
+---+-------------+-----------+--------+----------+---------+
|  1|Alice Johnson|Engineering|95000.00|2020-01-15|     true|
|  2|    Bob Smith|  Marketing|75000.00|2021-03-20|     true|
|  3|Charlie Brown|Engineering|88000.00|2019-07-10|     true|
|  4| Diana Prince|      Sales|82000.00|2022-05-12|     true|
|  5| Eve Anderson|Engineering|92000.00|2021-11-08|     true|
+---+-------------+-----------+--------+----------+---------+



In [105]:
if USE_SPARK and 'spark' in dir():
    
    try:        
        # 6: Perform aggregation
        print("\nStep 6: Aggregation - Average salary by department:")
        spark.sql(f"""
            SELECT 
                department,
                COUNT(*) as employee_count,
                AVG(salary) as avg_salary,
                MIN(hire_date) as earliest_hire
            FROM {demo_table_name}
            WHERE is_active = true
            GROUP BY department
            ORDER BY avg_salary DESC
        """).show()
        
        print("\nDemo complete! Table created at:", demo_table_path)
        
    except Exception as e:
        print(f"Error: {e}")
else:
    print("Spark not enabled. Set USE_SPARK = True in Section 6.")


Step 6: Aggregation - Average salary by department:
+-----------+--------------+------------+-------------+
| department|employee_count|  avg_salary|earliest_hire|
+-----------+--------------+------------+-------------+
|Engineering|             3|91666.666667|   2019-07-10|
|      Sales|             1|82000.000000|   2022-05-12|
|  Marketing|             1|75000.000000|   2021-03-20|
+-----------+--------------+------------+-------------+


Demo complete! Table created at: /Users/anh.nguyen/Documents/poc/deltalake_poc/deltalake/demo_employees


In [106]:
# Demo: MERGE (Upsert) operation
if USE_SPARK and 'spark' in dir():

    if os.path.exists(demo_table_path):
        print("=== Demo: MERGE (Upsert) Operation ===\n")
        
        try:
            # Load table
            df = spark.read.format("delta").load(demo_table_path)
            df.createOrReplaceTempView(demo_table_name)
            
            print("1: Performing MERGE operation...")
            print("  - Update Charlie's department and salary")
            print("  - Insert new employee Frank\n")
            
            spark.sql(f"""
                MERGE INTO {demo_table_name} AS target
                USING (
                    SELECT 3 as id, 'Charlie Brown' as name, 'Data Science' as department, 
                           98000.00 as salary, DATE '2019-07-10' as hire_date, true as is_active
                    UNION ALL
                    SELECT 6 as id, 'Frank Miller' as name, 'Sales' as department,
                           78000.00 as salary, DATE '2023-01-15' as hire_date, true as is_active
                ) AS source
                ON target.id = source.id
                WHEN MATCHED THEN UPDATE SET
                    target.department = source.department,
                    target.salary = source.salary
                WHEN NOT MATCHED THEN INSERT (id, name, department, salary, hire_date, is_active)
                    VALUES (source.id, source.name, source.department, source.salary, source.hire_date, source.is_active)
            """)
            
            print("✅ MERGE complete!\n")
            print("Changes:")
            print("   - Charlie: department 'Engineering' → 'Data Science', salary $88,000 → $98,000")
            print("   - Frank: NEW employee added\n")
            
            # View updated data
            print("Updated table:")
            df_merged = spark.read.format("delta").load(demo_table_path)
            df_merged.orderBy("id").show()
            
            print(f"\nTotal employees: {df_merged.count()}")
            
        except Exception as e:
            print(f"Error: {e}")
    else:
        print("Demo table not found. Run the 'Create Table' demo first.")
else:
    print("Spark not enabled.")

=== Demo: MERGE (Upsert) Operation ===

Step 1: Performing MERGE operation...
  - Update Charlie's department and salary
  - Insert new employee Frank

✅ MERGE complete!

📝 Changes:
   - Charlie: department 'Engineering' → 'Data Science', salary $88,000 → $98,000
   - Frank: NEW employee added

Updated table:
+---+-------------+------------+--------+----------+---------+
| id|         name|  department|  salary| hire_date|is_active|
+---+-------------+------------+--------+----------+---------+
|  1|Alice Johnson| Engineering|95000.00|2020-01-15|     true|
|  2|    Bob Smith|   Marketing|75000.00|2021-03-20|     true|
|  3|Charlie Brown|Data Science|98000.00|2019-07-10|     true|
|  4| Diana Prince|       Sales|82000.00|2022-05-12|     true|
|  5| Eve Anderson| Engineering|92000.00|2021-11-08|     true|
|  6| Frank Miller|       Sales|78000.00|2023-01-15|     true|
+---+-------------+------------+--------+----------+---------+


Total employees: 6


In [108]:
# Demo: UPDATE and DELETE operations
if USE_SPARK and 'spark' in dir():
    
    if os.path.exists(demo_table_path):
        print("=== Demo: UPDATE and DELETE Operations ===\n")
        
        try:
            # Load table
            df = spark.read.format("delta").load(demo_table_path)
            df.createOrReplaceTempView(demo_table_name)
            
            # Step 1: UPDATE records
            print("Step 1: Updating Alice's salary...")
            spark.sql(f"""
                UPDATE {demo_table_name}
                SET salary = 105000.00
                WHERE name = 'Alice Johnson'
            """)
            print("✅ Salary updated!\n")
            
            # Step 2: View updated data
            print("Step 2: Viewing updated record:")
            spark.sql(f"""
                SELECT id, name, department, salary
                FROM {demo_table_name}
                WHERE name = 'Alice Johnson'
            """).show()
            
            # Step 3: DELETE a record
            print("\nStep 3: Deactivating Bob (soft delete)...")
            spark.sql(f"""
                UPDATE {demo_table_name}
                SET is_active = false
                WHERE name = 'Bob Smith'
            """)
            print("✅ Record deactivated!\n")
            
            # Step 4: View active records only
            # print("Step 4: Viewing active employees only:")
            # spark.sql(f"""
            #     SELECT id, name, department, salary, is_active
            #     FROM {demo_table_name}
            #     WHERE is_active = true
            #     ORDER BY id
            # """).show()
            
            # Step 5: View table history
            print("\nStep 5: View Delta table history:")
            from delta import DeltaTable
            dt = DeltaTable.forPath(spark, demo_table_path)
            dt.history().select("version", "operation", "operationMetrics").show(truncate=False)
            
        except Exception as e:
            print(f"Error: {e}")
    else:
        print("Demo table not found. Run the 'Create Table' demo first.")
else:
    print("Spark not enabled.")

=== Demo: UPDATE and DELETE Operations ===

Step 1: Updating Alice's salary...
✅ Salary updated!

Step 2: Viewing updated record:
+---+-------------+-----------+---------+
| id|         name| department|   salary|
+---+-------------+-----------+---------+
|  1|Alice Johnson|Engineering|105000.00|
+---+-------------+-----------+---------+


Step 3: Deactivating Bob (soft delete)...
✅ Record deactivated!


Step 5: View Delta table history:
+-------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Spark: Stop session when done
if USE_SPARK and 'spark' in dir():
    # Uncomment to stop Spark session
    # spark.stop()
    # print("SparkSession stopped.")
    pass

In [22]:
# View schema for a table
TABLE_TO_INSPECT = "customers"  # Change to inspect different tables

if table_exists(TABLE_TO_INSPECT):
    schema = get_schema(TABLE_TO_INSPECT)
    print(f"Schema for '{TABLE_TO_INSPECT}' table")
    print("=" * 60)
    print(schema)
else:
    print(f"Table '{TABLE_TO_INSPECT}' not found.")

Schema for 'customers' table
id: int64
first_name: string
last_name: string
email: string
phone: string
created_at: timestamp[us, tz=UTC]
updated_at: timestamp[us, tz=UTC]
__cdc_operation: string
__cdc_timestamp: timestamp[us, tz=UTC]
__processed_at: timestamp[us, tz=UTC]


In [23]:
# View all table schemas
print("All Table Schemas")
print("=" * 60)

for table in TABLES:
    if table_exists(table):
        print(f"\n--- {table} ---")
        schema = get_schema(table)
        for field in schema:
            print(f"  {field.name}: {field.type}")

All Table Schemas

--- customers ---
  id: int64
  first_name: string
  last_name: string
  email: string
  phone: string
  created_at: timestamp[us, tz=UTC]
  updated_at: timestamp[us, tz=UTC]
  __cdc_operation: string
  __cdc_timestamp: timestamp[us, tz=UTC]
  __processed_at: timestamp[us, tz=UTC]

--- products ---
  id: int64
  name: string
  description: string
  price: double
  stock_quantity: int64
  category: string
  created_at: timestamp[us, tz=UTC]
  updated_at: timestamp[us, tz=UTC]
  __cdc_operation: string
  __cdc_timestamp: timestamp[us, tz=UTC]
  __processed_at: timestamp[us, tz=UTC]

--- orders ---
  id: int64
  customer_id: int64
  order_date: timestamp[us, tz=UTC]
  status: string
  total_amount: double
  shipping_address: string
  created_at: timestamp[us, tz=UTC]
  updated_at: timestamp[us, tz=UTC]
  __cdc_operation: string
  __cdc_timestamp: timestamp[us, tz=UTC]
  __processed_at: timestamp[us, tz=UTC]

--- order_items ---
  id: int64
  order_id: int64
  product_id

In [24]:
# Custom query template
# Modify the TABLE and add your own filters

TABLE = "customers"  # Change table name

if table_exists(TABLE):
    df = load_table(TABLE)
    
    # Add your filters here using pandas
    # Example: df_filtered = df[df['id'] > 5]
    # Example: df_filtered = df[df['name'].str.contains('John')]
    
    display(df)
else:
    print(f"Table '{TABLE}' not found.")

,id,first_name,last_name,email,phone,created_at,updated_at,__cdc_operation,__cdc_timestamp,__processed_at
0,5,Charlie,Brown,charlie.brown@example.com,+1-555-0105,1970-01-21 10:32:19+00:00,1970-01-21 10:32:19+00:00,r,2025-12-17 02:53:47+00:00,2025-12-17 03:32:39+00:00
1,4,Alice,Williams,alice.williams@example.com,+1-555-0104,1970-01-21 10:32:19+00:00,1970-01-21 10:32:19+00:00,r,2025-12-17 02:53:47+00:00,2025-12-17 03:32:38+00:00
2,3,Bob,Johnson,bob.johnson@example.com,+1-555-0103,1970-01-21 10:32:19+00:00,1970-01-21 10:32:19+00:00,r,2025-12-17 02:53:47+00:00,2025-12-17 03:32:36+00:00
3,2,Jane,Smith,jane.smith@example.com,+1-555-0102,1970-01-21 10:32:19+00:00,1970-01-21 10:32:19+00:00,r,2025-12-17 02:53:47+00:00,2025-12-17 03:32:32+00:00
4,1,John,Doe,john.doe@example.com,+1-555-0101,1970-01-21 10:32:19+00:00,1970-01-21 10:32:19+00:00,r,2025-12-17 02:53:47+00:00,2025-12-17 03:32:27+00:00


In [ ]:
# Export query results to CSV
# Uncomment and modify as needed

# TABLE = "customers"
# OUTPUT_FILE = "./query_results.csv"

# if table_exists(TABLE):
#     df = load_table(TABLE)
#     df.to_csv(OUTPUT_FILE, index=False)
#     print(f"Exported {len(df)} rows to {OUTPUT_FILE}")

---
## Summary

This notebook demonstrates two approaches to querying Delta Lake:

### 1. **delta-rs (Python-native)**
- **Sections:** 2-5
- **Library:** `deltalake` Python package
- **Pros:** 
  - Fast, lightweight (no JVM)
  - Perfect for read-only operations
  - Time travel support
- **Cons:** 
  - Limited to reads, history, and metadata
  - No SQL support
  - No write operations
- **Use when:** You need fast queries and don't need SQL or writes

### 2. **PySpark SQL**
- **Sections:** 6-6.1
- **Library:** `pyspark` with Delta Lake
- **Pros:** 
  - Full SQL support (SELECT, INSERT, UPDATE, DELETE, MERGE)
  - Complex transformations and joins
  - Schema evolution
  - Write operations
- **Cons:** 
  - Heavier (requires JVM)
  - Python 3.14 requires SQL approach (not DataFrame API)
- **Use when:** You need SQL queries, writes, or complex operations

### Quick Reference

**delta-rs:**
```python
# Load table (current version)
df = load_table("customers")

# Load table (specific version)
df = load_table("customers", version=0)

# Get table history
history = get_history("customers")

# Check if table exists
exists = table_exists("customers")
```

**PySpark SQL:**
```python
# Query with SQL
result = spark.sql("SELECT * FROM customers WHERE city = 'NYC'")

# Time travel
df = spark.read.format("delta").option("versionAsOf", 0).load(path)

# Write operations
spark.sql("INSERT INTO customers VALUES (...)")
spark.sql("MERGE INTO customers ...")
```